In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn import datasets

import keras

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


### Import dataset

In [2]:
train_ds = tfds.load(name="mnist", split=tfds.Split.TRAIN, shuffle_files=True)
test_ds = tfds.load(name="mnist", split=tfds.Split.TEST, shuffle_files=True)

tfds.as_numpy(test_ds)
X_train = (t['image'].reshape(784) for t in tfds.as_numpy(train_ds))
y_train = (t['label'] for t in tfds.as_numpy(train_ds))
X_test = (t['image'].reshape(784) for t in tfds.as_numpy(test_ds))
y_test = (t['label'] for t in tfds.as_numpy(test_ds))

# Eagerly fetch whole dataset
X_train = list(X_train)
y_train = list(y_train)
X_test = list(X_test)
y_test = list(y_test)

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

mlpc = MLPClassifier(hidden_layer_sizes=(512), learning_rate='adaptive')
clf = Pipeline([('scaler', StandardScaler()), ('gp', mlpc)]) 
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('gp',
                 MLPClassifier(activation='relu', alpha=0.0001,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=512, learning_rate='adaptive',
                               learning_rate_init=0.001, max_iter=200,
                               momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='adam',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False)

In [11]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

y_train_vec = np.zeros((len(y_train), 10))
y_train_vec[np.arange(len(y_train)), y_train] = 1

model.fit(X_train, y_train_vec, epochs=1)


KeyboardInterrupt: 

## GP Classification

In [177]:
%%time
kernel = RBF()
gpc = GaussianProcessClassifier(kernel, random_state=23, n_restarts_optimizer=10)
gpc.fit(X_train, y_train)
print('Accuracy: %.8f, kernel hyperparameters:%s, logevidence: %.2f ' % (np.mean(np.equal(y_test, gpc.predict(X_test))), gpc.kernel_, gpc.log_marginal_likelihood_value_))

Accuracy: 0.95522388, kernel hyperparameters:RBF(length_scale=26.7), logevidence: -9.04 
CPU times: user 110 ms, sys: 50 ms, total: 160 ms
Wall time: 91.9 ms


In [174]:
pd.DataFrame(gpc.predict(X_train)).loc[:,0].value_counts()

0    19
8    16
Name: 0, dtype: int64

In [250]:
np.mean(y_test==0)

0.9033957845433255

In [243]:
d = pd.DataFrame(y_train, columns=['abc'])
d.abc.value_counts()

False    80
True      9
Name: abc, dtype: int64

In [ ]:


a = 5
set_trace()
b = a+a
c = 3

--Return--
None
> <ipython-input-3-cbb1983ad13f>(4)<module>()
      2 
      3 a = 5
----> 4 set_trace()
      5 b = a+a
      6 c = 3

--KeyboardInterrupt--


ipdb>  c
ipdb>  c


In [6]:
%debug

> <ipython-input-3-06bf57bc54d0>(1)<module>()
----> 1 5/a
      2 np.zeros(10)



ipdb>  n
